In [1]:
import numpy as np

In [2]:
import cv2

In [3]:
import os

In [4]:
import matplotlib.pyplot as pt

In [5]:
import mediapipe as mp

In [10]:
import pandas as pd

In [6]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

# Feature Extraction

In [7]:
def extraction(results):
    # face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    poses = np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
    lh = np.array([[res.x,res.y,res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(63)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(63)
    return np.concatenate([poses, lh , rh])
    

In [8]:
def draw(image , results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION)
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.pose_landmarks,mp_holistic.POSE_CONNECTIONS)

In [9]:
def detectVideo(image, model):
    results = model.process(image)
    return image, results

In [11]:
df = pd.read_csv('dataset.csv')

In [12]:
selected = df[['word_id', 'word', 'sentence_id','sentence']]


In [20]:

mappings = dict(zip(selected['word'][:73], selected['word_id'][:73]))


In [21]:
filtered = {'concept': [7, 30],
 'attach': [0, 25],
 'meter': [2, 25],
 'owing to': [7, 30],
 'advertisement': [0, 26],
 'bouquet': [7, 30],
 'altogether': [3, 28],
 'concerned': [3, 29],
 'unbreakable': [2, 30],
 'accomplish': [16, 30],
 'interaction': [2, 28],
 'electricity': [4, 30],
 'maker': [2, 25],
 'impel': [2, 29],
 'acquisition': [2, 29],
 'elude - 2': [2, 28],
 'court': [0, 28],
 'persist': [4, 29],
 'advantage': [0, 31],
 'buddy': [0, 28],
 'adversarial': [6, 30],
 'accurate': [0, 29],
 'queue': [0, 30],
 'accent': [2, 25],
 'adapt': [6, 30],
 'enthusiasm': [7, 30],
 'tender': [2, 26],
 'awareness': [8, 30],
 'frustration': [3, 30],
 'compel': [0, 31],
 'rude': [4, 30],
 'policy': [7, 30],
 'secured': [2, 30],
 'elude - 1': [2, 25],
 'everywhere': [3, 29],
 'advert': [5, 30],
 'effective': [0, 22],
 'accompany': [2, 25],
 'adopt': [6, 30],
 'procrastinate': [7, 30],
 'consultant': [2, 24],
 'exceed': [3, 28],
 'stupid': [0, 26],
 'examination': [2, 27],
 'mixture': [2, 30],
 'consensus': [0, 31],
 'candidate': [0, 31],
 'decorate': [3, 29],
 'menu': [2, 25],
 'zygote': [0, 28],
 'decisive': [3, 30],
 'shrivel': [4, 30],
 'analysis': [2, 29],
 'edgy': [2, 25],
 'rancour': [2, 28]}

# Model

In [121]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [122]:
label_map = {label:num for num , label in enumerate(filtered)}
labels = []

In [123]:
for val in filtered: 
    labels.append(label_map[val])

In [125]:
sequences = []
label_map = {}
words = []
for val in result: 
    words.append(val)
    window = []
    for frame_num in range(30):
        res = None
        try:
            res = np.load(os.path.join("DATA" , val , "{}.npy".format(frame_num + 1)))
        except:
            res = window[-1]
        window.append(res)
    sequences.append(window)
    

IndexError: list index out of range

In [33]:
x = np.array(sequences)

In [34]:
y = to_categorical(labels).astype(int)

# LSTM

In [116]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import TensorBoard


In [117]:
log_dir = os.path.join('logs')
tb_callback = TensorBoard(log_dir = log_dir)

In [118]:
model = Sequential()

In [119]:
model.add(LSTM(units=128, input_shape=(30 , 258 ) , return_sequences=True))
model.add(LSTM(units=128, return_sequences=True))
model.add(LSTM(units=64))
model.add(Dense(units=256, activation='relu'))
# model.add(Dropout(0.5))  # Adding dropout for regularization
model.add(Dense(units=55, activation='softmax'))


In [120]:
model.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=['categorical_accuracy'])

In [102]:
 model.fit(x, y , epochs=2000,callbacks=[tb_callback])

Epoch 1/2000
2/2 [==============================] - 10s 1s/step - loss: 4.0493 - categorical_accuracy: 0.0182
Epoch 2/2000
2/2 [==============================] - 0s 73ms/step - loss: 4.0205 - categorical_accuracy: 0.0000e+00
Epoch 3/2000
2/2 [==============================] - 0s 71ms/step - loss: 3.9816 - categorical_accuracy: 0.0182
Epoch 4/2000
2/2 [==============================] - 0s 67ms/step - loss: 3.9372 - categorical_accuracy: 0.0545
Epoch 5/2000
2/2 [==============================] - 0s 70ms/step - loss: 3.8940 - categorical_accuracy: 0.0545
Epoch 6/2000
2/2 [==============================] - 0s 70ms/step - loss: 3.8341 - categorical_accuracy: 0.0727
Epoch 7/2000
2/2 [==============================] - 0s 68ms/step - loss: 3.7646 - categorical_accuracy: 0.0909
Epoch 8/2000
2/2 [==============================] - 0s 69ms/step - loss: 3.7263 - categorical_accuracy: 0.0727
Epoch 9/2000
2/2 [==============================] - 0s 69ms/step - loss: 3.6391 - categorical_accuracy: 0.109

KeyboardInterrupt: 

In [103]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_10 (LSTM)              (None, 30, 128)           198144    
                                                                 
 lstm_11 (LSTM)              (None, 30, 128)           131584    
                                                                 
 lstm_12 (LSTM)              (None, 64)                49408     
                                                                 
 dense_6 (Dense)             (None, 256)               16640     
                                                                 
 dense_7 (Dense)             (None, 55)                14135     
                                                                 
Total params: 409911 (1.56 MB)
Trainable params: 409911 (1.56 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


# testing the training data : >

In [104]:
error = 0
correct = 0

for i in range(x.shape[0]):
    temp = model.predict(np.expand_dims(x[i], axis = 0))[0]
    predicted = words[np.argmax(temp)]
    ans = words[np.argmax(y[i])]
    if ans != predicted:
        error += 1
    else:
        correct += 1

1/1 [==============================] - 0s 14ms/step


In [105]:
print(correct / (correct + error))

1.0


# TESTING ON RECORDED VIDEOS

In [126]:
testlist = ["advertisement","rude","concept","buddy","bouquet"]

In [107]:
maxFrameCount = -1
maxWord = ""
frame_sizes = {}
# testlist = ["advertisement","rude","concept","electricity" , "buddy" , "maker"]
with mp_holistic.Holistic(min_detection_confidence = 0.5 , min_tracking_confidence = 0.9) as mp_model:
    
    
    for val in testlist:
        # if val not in filtered:
        
        i = 0
        
        reader = cv2.VideoCapture(os.path.join('test' , val + '.mp4'))
        # reader = cv2.VideoCapture(os.path.join('ISL-videos' , mappings[val] + '_left__+'.mp4'))
        # print(val + " " + str(reader.get(cv2.CAP_PROP_FRAME_COUNT)))
        curFrameLen = int(reader.get(cv2.CAP_PROP_FRAME_COUNT))
        # print(val + " " + str(curFrameLen))
        
        for frame_no in range(120):
            # print(frame_no)
            # if frame_no % 2 == 0:
               
            ret , frame = reader.read()
            # print(ret == None)

            # if frame_no % 2 != 0:
            #     continue
            # else:
            if ret:
                i+=1
                # if (frame_no + 1) % 5 == 0: 
                # npy_path = os.path.join(PATH,val, str(i))
                image, results = detectVideo(frame, mp_model)
                draw(image, results)
                # keypoints = extraction(results)
                # np.save(npy_path, keypoints)
                cv2.imshow('Feed', image)
                # i += 1
            else:
                pass
                # keypoints = np.zeros(features)
                # npy_path = os.path.join(PATH,val, str(i))
                # i+=1
                # np.save(npy_path, keypoints)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
        # print( val , mappings[val])
        print(val +" "+ str(i))
        # temp = input("next ? ")
        # if temp == 'y':
        frame_sizes[val] = i
        if val == 'toil':
            break
    reader.release() 
    cv2.destroyAllWindows()
            # continue


advertisement 60
rude 58
concept 37
buddy 50
boquet 60


In [108]:
print(frame_sizes)

{'advertisement': 60, 'rude': 58, 'concept': 37, 'buddy': 50, 'boquet': 60}


In [109]:
result = dry_run(frame_sizes)

initial = 60 ----> 30 (2) (advertisement)
initial = 58 ----> 29 (2) (rude)
initial = 37 ----> 30 (5) (concept)
initial = 50 ----> 25 (2) (buddy)
initial = 60 ----> 30 (2) (boquet)
0


In [110]:
print(result)

{'advertisement': [2, 30], 'rude': [2, 29], 'concept': [5, 30], 'buddy': [2, 25], 'boquet': [2, 30]}


In [96]:
maxFrameCount = -1
maxWord = ""
with mp_holistic.Holistic(min_detection_confidence = 0.5 , min_tracking_confidence = 0.9) as mp_model:
    
    
    for val in result:
        
        try: 
            os.makedirs( os.path.join(PATH,val) )
        except:
            pass
        
        i = 1
        dropped = 0
        captured = 0
        
        # reader = cv2.VideoCapture(os.path.join('ISL-videos' , mappings[val] + '.mp4'))
        reader = cv2.VideoCapture(os.path.join('test' , val + '.mp4'))
        # print(val + " " + str(reader.get(cv2.CAP_PROP_FRAME_COUNT)))
        # curFrameLen = int(reader.get(cv2.CAP_PROP_FRAME_COUNT))
        # print(val + " " + str(curFrameLen))
        to_skip = result[val][0]
        # print("to drop " ) 
        # print(to_skip)
        for frame_no in range(120):
            # print(frame_no)
            # if frame_no % 2 == 0:
               
            ret , frame = reader.read()
            # print(ret == None)

            # if frame_no % 2 != 0:
            #     continue
            # else:
            if ret:
                if (to_skip > 0) and ((i%to_skip) == 0):
                    # print("HI")
                    i+=1
                    dropped += 1
                    continue
                else:
                    i+=1
                    captured +=1 
                    # if (frame_no + 1) % 5 == 0: 
                    npy_path = os.path.join("test_frames",val, str(captured))
                    image, results = detectVideo(frame, mp_model)
                    draw(image, results)
                    keypoints = extraction(results)
                    # print(keypoints.shape)
                    np.save(npy_path, keypoints)
                    cv2.imshow('Feed', image)
                # i += 1
            else:
                pass
                # keypoints = np.zeros(features)
                # npy_path = os.path.join(PATH,val, str(i))
                # i+=1
                # np.save(npy_path, keypoints)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
        # print( val , mappings[val])
        print(val)
        print(f'captured = {captured}')
        # print(f'expected {frame_sizes[val]}')
        # print(f'calculated {filtered[val][1]}')
        print(f'dropped = {dropped}')
        # print(f'work = {filtered[val]}')
        print("------------------------------------")
        
        # temp = input("next ? ")
        # if temp == 'y':
        # frame_sizes[val] = i
        reader.release()
        cv2.destroyAllWindows()
            # continue


advertisement
captured = 30
dropped = 30
------------------------------------
rude
captured = 29
dropped = 29
------------------------------------
concept
captured = 30
dropped = 7
------------------------------------
buddy
captured = 25
dropped = 25
------------------------------------
boquet
captured = 30
dropped = 30
------------------------------------


In [111]:
sequences = []
# label_map = {}
# words = []
for val in result: 
    words.append(val)
    window = []
    for frame_num in range(30):
        res = None
        try:
            res = np.load(os.path.join("test_frames" , val , "{}.npy".format(frame_num + 1)))
        except:
            res = window[-1]
        window.append(res)
    sequences.append(window)
    

In [112]:
print(len(sequences))

5


In [113]:
x_test = np.array(sequences)

In [114]:
print(testlist)


['advertisement', 'rude', 'concept', 'buddy', 'boquet']


In [115]:
error = 0
correct = 0

for i in range(x_test.shape[0]):
    temp = model.predict(np.expand_dims(x_test[i], axis = 0))[0]
    predicted = words[np.argmax(temp)]
    print(predicted)
    print(testlist[i])
    # ans = words[np.argmax(y_train[i])]
    ans = testlist[i]
    if ans != predicted:
        error += 1
    else:
        correct += 1

1/1 [==============================] - 0s 18ms/step
accompany
advertisement
1/1 [==============================] - 0s 14ms/step
interaction
rude
1/1 [==============================] - 0s 14ms/step
compel
concept
1/1 [==============================] - 0s 14ms/step
exceed
buddy
1/1 [==============================] - 0s 14ms/step
accompany
boquet
